### Task: Implement Bloom filter and test it on two animal lists given below

In [1]:
animals = ['dog', 'cat', 'giraffe', 'fly', 'mosquito', 'horse', 'eagle',
           'bird', 'bison', 'boar', 'butterfly', 'ant', 'anaconda', 'bear',
           'chicken', 'dolphin', 'donkey', 'crow', 'crocodile']

other_animals = ['badger', 'cow', 'pig', 'sheep', 'bee', 'wolf', 'fox',
                 'whale', 'shark', 'fish', 'turkey', 'duck', 'dove',
                 'deer', 'elephant', 'frog', 'falcon', 'goat', 'gorilla',
                 'hawk']

In [2]:
!pip install mmh3
import mmh3
from random import randint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 2.8 MB/s eta 0:00:00


In [3]:
class BloomFilter():
  def __init__(self, L, hash_num):
    self.L = L #length of bitmap
    self.hash_num = hash_num #number of hash functions
    self.B = [0 for i in range(self.L)] #bitmap
    self.seeds = [randint(100, 1000) for _ in range(self.hash_num)]
    self.H = [lambda s: mmh3.hash(s, seed) for seed in self.seeds] #list of hash functions

  def add(self, key): #add element to set
    for h in self.H:
      self.B[h(key) % self.L] = 1

  def isin(self, key): #check if element is in the set
    for h in self.H:
      if self.B[h(key) % self.L] == 0:
        return False
    return True

Bloom filter characterizes itself with high space efficiency and no false negatives (meaning that if element isn't in the set, then bloom filter will detect that with $100\%$ accuracy). It has capacity for false postives (if element isn't in the set, then bloom filter *might* detect it as element in the set). Therefore, bloom filter excels at detecting if element is *definitely not* in set, and you can tolerate a bit of false positives.

In [4]:
bloom = [BloomFilter(L = 100, hash_num = 4), BloomFilter(L = 100, hash_num = 20),
         BloomFilter(L = 500, hash_num = 4), BloomFilter(L = 500, hash_num = 20)]

for animal in animals:
  for b in bloom:
    b.add(animal)

In [5]:
FP = [0 for i in range(len(bloom))]
for n, b in enumerate(bloom):
  print(f"\n{n+1}. L = {b.L}, hash_num = {b.hash_num}")
  for other in other_animals:
    if b.isin(other):
      FP[n] += 1
      print(other)
  print(f'FP = {FP[n]}, FPR = {100*FP[n]/len(other_animals)}%')


1. L = 100, hash_num = 4
badger
fish
gorilla
FP = 3, FPR = 15.0%

2. L = 100, hash_num = 20
cow
wolf
shark
fish
FP = 4, FPR = 20.0%

3. L = 500, hash_num = 4
FP = 0, FPR = 0.0%

4. L = 500, hash_num = 20
FP = 0, FPR = 0.0%


In [6]:
#more optimal - use bitarray, since bitmap only has binary values
!pip install bitarray
from bitarray import bitarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 7.3 MB/s eta 0:00:00


In [7]:
class BloomFilter():
  def __init__(self, L, hash_num):
    self.L = L
    self.hash_num = hash_num
    self.B = bitarray(self.L)
    self.B.setall(0)
    self.seeds = [randint(100, 1000) for _ in range(self.hash_num)]
    self.H = [lambda s: mmh3.hash(s, seed) for seed in self.seeds]

  def add(self, key):
    for h in self.H:
      self.B[h(key) % self.L] = 1

  def isin(self, key):
    for h in self.H:
      if self.B[h(key) % self.L] == 0:
        return False
    return True

In [8]:
bloom = [BloomFilter(L = 100, hash_num = 4), BloomFilter(L = 100, hash_num = 20),
         BloomFilter(L = 500, hash_num = 4), BloomFilter(L = 500, hash_num = 20)]

for animal in animals:
  for b in bloom:
    b.add(animal)

In [9]:
FP = [0 for i in range(len(bloom))]
for n, b in enumerate(bloom):
  print(f"\n{n+1}. L = {b.L}, hash_num = {b.hash_num}")
  for other in other_animals:
    if b.isin(other):
      FP[n] += 1
      print(other)
  print(f'FP = {FP[n]}, FPR = {100*FP[n]/len(other_animals)}%')


1. L = 100, hash_num = 4
wolf
turkey
goat
FP = 3, FPR = 15.0%

2. L = 100, hash_num = 20
dove
frog
FP = 2, FPR = 10.0%

3. L = 500, hash_num = 4
frog
FP = 1, FPR = 5.0%

4. L = 500, hash_num = 20
elephant
frog
FP = 2, FPR = 10.0%


Number of false positives is random, because it depends on how seed of hash function. We can however estimate (if hash functions are independent) false positive rate. Approximately, this will be equal to:
\begin{equation}
FPR \approx (1 - e^{\frac{-kn}{L}})^k,
\end{equation}
where, $k$ is number of hash functions, $n$ is  number of elements in set and $L$ is length of bitmap. In these 4 examples this probability is equal to

1.   $FPR = 0.08$
2.   $FPR = 0.64$
3.   $FPR = 0.0004$
4.   $FPR = 3.34*10^{-6}$

These approximations might not be accurate here, because of small numbers ($19$ elements in set is laughably small).